# Understanding Apple Quality: A Prediction Based on Ripeness and Acidity
### by Paul Bunuan, Nathan Ng, Tyler Tan, Junyi Yao

# Introduction
    Apples are a common fruit consumed by people every day, with a variety of species being cultivated and harvested from farms across the world. In 2014, roughly 84 million tonnes of apples were produced (Musacchi & Serra, 2018). It is important for producers and sellers to maintain a quality standard for apples with regards to characteristics such as acidity, colour, shape, weight, and more. The values of these qualities will influence sales and the consumer’s chances of purchasing the product again. There are different ways manufacturers can measure the traits of apples. For instance, as apples ripen, they tend to become more alkaline with decreasing malic acid content, the primary acid found in apples. (Farcuh, 2018). However, the acidity is not used to track apple maturity because of the variation amongst species and differing optimal acid content for consumption (Farcuh, 2018). In our project, we aim to determine how the acidity and ripeness values of any given apple influences its overall quality status as “good” or “bad”; to answer this predictive question, we use a public dataset from Kaggle regarding apple quality (Elgiriyewithana, 2024). 

    	This dataset contains nine columns, which include Apple ID, Size, Weight, Sweetness, Crunchiness, Juiciness, Ripeness, Acidity, and Quality. However, for the purposes of this report, only the Ripeness, Acidity, and Quality columns will be used. It is hypothesized that apples with a higher Ripeness value and lower Acidity value are more likely to be characterized as “good” quality apples.


# Methods & Results
## Methods
1. describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.2. 
your report should include code which:- 
loads data from the original source on the web-  
wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analys- is
performs a summary of the data set that is relevant for exploratory data analysis related to the planned analys- is 
creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned anal- ysis
performs the data ana- lysis
creates a visualization of the aa- lysis 
note: all tables and figure should have a figure/table number and a
Here's a sentence with a footnote. [^1]

[^1]: This is the footnote. legend


# Disscusion
summarize what you found
discuss whether this is what you expected to find?
discuss what impact could such findings have?
discuss what future questions could this lead to?


# Sources 
Elgiriyewithana, N. (2024, January 11). Apple Quality. <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp
;Kaggle.  https://www.kaggle.com/datasets/nelgiriyewithana/apple-quality/data 
    
Farcuh, M. (2023). Fruit Harvest - Determining Apple Fruit Maturity and Optimal Harvest Date. Determining Apple Fruit Maturity and Optimal Harvest Date. 

    Musacchi, S., & Serra, S. (2018). Apple Fruit Quality: Overview on pre-harvest factors. Scientia Horticulturae, 234, 409–430. https://doi.org/10.1016/j.scienta.2017.12.057 